In [74]:
from utils.load_dataset import load_dataset, read_compressed_data

include_predictions = True

if include_predictions:
    groundtruth_index = 38
else:
    groundtruth_index = 6

data = load_dataset(include_predictions=include_predictions)

# data_2 = read_compressed_data("data/processed_data/arrival_data/bus_1_38.json.gz")


In [75]:
print(data[:5])

print(len(data[1]))

[[0.00414518376965307, 0.026042013301566348, 0.9996608492599883, 106, 0.00016, 4, 0.04258, 0.05319, 0.05622, 0.05774, 0.06077, 0.0638, 0.06531, 0.06834, 0.00168, 0.00168, 0.00471, 0.00471, 0.00622, 0.00774, 0.00925, 0.01077, 0.01228, 0.01228, 0.0138, 0.01531, 0.01531, 0.01683, 0.01834, 0.01986, 0.02289, 0.0244, 0.02592, 0.02895, 0.03198, 0.03501, 0.03652, 0.03955, {10: 0.04427, 2: 0.0461, 5: 0.04794, 52: 0.04824, 41: 0.05252, 20: 0.05313, 108: 0.05435, 106: 0.002, 34: 0.00443, 101: 0.00534, 47: 0.00625, 100: 0.00807, 102: 0.00928, 105: 0.01081, 69: 0.01415, 139: 0.01446, 136: 0.01538, 130: 0.01598, 129: 0.0166, 140: 0.01782, 133: 0.01964, 135: 0.02055, 138: 0.02146, 97: 0.02329, 66: 0.0242, 63: 0.02572, 42: 0.02754, 98: 0.02997, 38: 0.03028, 39: 0.03301, 72: 0.03481, 43: 0.03968}], [0.00414518376965307, 0.026042013301566348, 0.9996608492599883, 106, 0.00046, 4, 0.04288, 0.05349, 0.05652, 0.05804, 0.06107, 0.0641, 0.06561, 0.06864, 0.00198, 0.00198, 0.00501, 0.00501, 0.00652, 0.00804, 0

In [76]:
len(data)

# WITHOUT ZEROES:
# 125823
# WITH ZEROES:
# 126478

126478

In [77]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Preparing the input and output data
X = []
Y = []

for entry in data:
    try:
        features = entry[:groundtruth_index]
        targets = [entry[groundtruth_index][stop] for stop in entry[groundtruth_index]]
        X.append(features)
        Y.append(targets)
    except Exception as e:
        print(e)
        print(entry)

X = np.array(X)
Y = np.array(Y)

# Splitting data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Model definition and training
model = MultiOutputRegressor(LinearRegression())
model.fit(X_train, y_train)

# Prediction and performance evaluation
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mean_distance = mae * 11 * 60
print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mae}")
print(f"Mean Distance: {mean_distance}")

# WITH THE DOWNTOWNER APP:
# WITH ZEROES:
# Mean Squared Error: 0.0012367148906002779
# Mean Absolute Error: 0.011857285328340308
# Mean Distance: 7.825808316704604

# WITHOUT THE DOWNTOWNER APP:
# Mean Squared Error: 0.0013448742694995505
# Mean Absolute Error: 0.01531571679849884
# Mean Distance: 10.108373087009234

# DOWNTOWNER APP ONLY:
# Average MSE: 0.0015
# Average MAE: 0.0115
# Average Distance: 7.6006
# Given that this is lower than the linear regression using this, it's probably because we're optimizing MSE with linear regression instead of MAE. This gives us a lower MSE but not a lower MAE. But we care about MAE! So linear regression is probably not the best to use. 


Mean Squared Error: 0.0012367148906002779
Mean Absolute Error: 0.011857285328340308
Mean Distance: 7.825808316704604


In [51]:
# Example prediction
example_data = np.array([[0.00414518376965307, 0.026042013301566348, 0.9996608492599883, 106, 0.00016, 4]])
predicted_arrivals = model.predict(example_data)
print("Predicted Arrivals:", predicted_arrivals)

Predicted Arrivals: [[0.0321973  0.03649701 0.04329278 0.04818093 0.05999125 0.07099191
  0.07210766 0.04162347 0.03089792 0.01840754 0.01472464 0.00722708
  0.00444549 0.00495059 0.01749649 0.01925287 0.01892156 0.02216989
  0.0245471  0.02612296 0.02882761 0.03062105 0.03117897 0.03538327
  0.04009627 0.0494207  0.05130089 0.04319462 0.04428828 0.04482223
  0.03875889 0.02564455]]


In [78]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split

class MyDataset(Dataset):
    def __init__(self, data):
        self.inputs = [torch.tensor(d[:groundtruth_index], dtype=torch.float32) for d in data]
        self.targets = [torch.tensor(list(d[groundtruth_index].values()), dtype=torch.float32) for d in data]

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return self.inputs[idx], self.targets[idx]

dataset = MyDataset(data)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=10)


In [79]:
import torch.nn as nn

class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(groundtruth_index, 64)  # Input to hidden layer
        self.fc2 = nn.Linear(64, 256)
        self.fc3 = nn.Linear(256, 1024)
        self.fc4 = nn.Linear(1024, 256)
        self.fc5 = nn.Linear(256, 32)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        x = self.fc5(x)
        return x

model = SimpleNet()


In [80]:
import torch.optim as optim

criterion = torch.nn.L1Loss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 75
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    # if epoch % 10 == 0, then halve the learning rate
    if epoch % 10 == 0:
        for g in optimizer.param_groups:
            g['lr'] = g['lr'] / 2
    print(f'Epoch {epoch+1}, Loss: {total_loss/len(train_loader)}')

# save the model
torch.save(model.state_dict(), 'simple_net.pth')


Epoch 1, Loss: 0.01807326370310316
Epoch 2, Loss: 0.011017653389889734
Epoch 3, Loss: 0.010497423904595527
Epoch 4, Loss: 0.009901231086807115
Epoch 5, Loss: 0.009676614414456186
Epoch 6, Loss: 0.009505706962385958
Epoch 7, Loss: 0.009416752458148847
Epoch 8, Loss: 0.009291536563478925
Epoch 9, Loss: 0.009209159320556838
Epoch 10, Loss: 0.009058709388792262
Epoch 11, Loss: 0.009059163805641096
Epoch 12, Loss: 0.008240318464296391
Epoch 13, Loss: 0.008199998892600561
Epoch 14, Loss: 0.008158800481495658
Epoch 15, Loss: 0.008147182506417548
Epoch 16, Loss: 0.008140621953778692
Epoch 17, Loss: 0.008116618483077449
Epoch 18, Loss: 0.00811570662538854
Epoch 19, Loss: 0.008117796364606196
Epoch 20, Loss: 0.008055005194050509
Epoch 21, Loss: 0.008076381067327448
Epoch 22, Loss: 0.0077106850449084555
Epoch 23, Loss: 0.007707843937366142
Epoch 24, Loss: 0.007673162928794277
Epoch 25, Loss: 0.007661378544162838
Epoch 26, Loss: 0.007667191409290445
Epoch 27, Loss: 0.007652030841097028
Epoch 28, L

In [81]:
model.eval()
total_test_loss = 0
average_distance = 0

with torch.no_grad():
    total_elements = 0  # To account for total number of predictions across all batches
    for inputs, targets in test_loader:
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        total_test_loss += loss.item()

        # Calculate average distance
        for i in range(outputs.shape[0]):
            for j in range(outputs.shape[1]):
                average_distance += abs(outputs[i, j].item() - targets[i, j].item())
                total_elements += 1

    print(f'Test Loss: {total_test_loss/len(test_loader)}')

average_distance /= total_elements
print("Average distance: ", average_distance * 60 * 11)

# 75 epochs, halving learning rate every 10 epochs, not using their estimates
# Test Loss: 0.007547942571261007
# Average distance:  4.981981012612223

# 75 epochs, halving learning rate every 10 epochs, using their estimates
# Test Loss: 0.0070760076254948595
# Average distance:  4.670307582987945


Test Loss: 0.0070760076254948595
Average distance:  4.670307582987945


In [18]:
from utils.path_conversion import find_progress

route_stops = [10, 2, 5, 52, 41, 20, 108, 106, 34, 101, 47, 100, 102, 105, 69, 139, 136, 130, 129, 140, 133, 135, 138, 97, 66, 63, 42, 98, 38, 39, 72, 43]

stops_info = [{"id":1,"name":"100 Church Street South","description":"","lat":41.299671,"lon":-72.929425},{"id":2,"name":"129 York","description":"","lat":41.306539,"lon":-72.933001},{"id":3,"name":"130 Prospect Street (N)","description":"","lat":41.315318,"lon":-72.924471},{"id":4,"name":"130 Prospect Street (S)","description":"","lat":41.315161,"lon":-72.924729},{"id":5,"name":"180 York (A&A)","description":"","lat":41.30872,"lon":-72.931417},{"id":6,"name":"205 Whitney Ave","description":"","lat":41.316293,"lon":-72.920383},{"id":7,"name":"221 Whitney (N)","description":"","lat":41.31685,"lon":-72.920112},{"id":8,"name":"300 Cedar (TAC)","description":"","lat":41.301758,"lon":-72.933542},{"id":9,"name":"300 George St","description":"","lat":41.305166,"lon":-72.930745},{"id":10,"name":"333 Cedar","description":"","lat":41.303254,"lon":-72.934247},{"id":11,"name":"344 Winchester","description":"","lat":41.324661,"lon":-72.928677},{"id":12,"name":"81 George St","description":"","lat":41.303464,"lon":-72.926405},{"id":13,"name":"Amistad / Cedar","description":"","lat":41.300236,"lon":-72.932021},{"id":14,"name":"Amistad / Church St South","description":"","lat":41.300531,"lon":-72.929866},{"id":15,"name":"Amistad Building / Garage","description":"","lat":41.300247,"lon":-72.931898},{"id":16,"name":"Amistad CSS Weekend Blue","description":"","lat":41.300614,"lon":-72.929956},{"id":17,"name":"Amistand / Cedar Weekend Blue","description":"","lat":41.300259,"lon":-72.932555},{"id":18,"name":"Ashmun / Webster","description":"","lat":41.317455,"lon":-72.929469},{"id":19,"name":"Audubon / Orange","description":"","lat":41.311054,"lon":-72.920135},{"id":20,"name":"Becton / 15 Prospect","description":"","lat":41.312609,"lon":-72.925331},{"id":21,"name":"Broadway / York","description":"","lat":41.311002,"lon":-72.930344},{"id":22,"name":"Building 400","description":"","lat":41.255793,"lon":-72.993569},{"id":23,"name":"Building 600","description":"","lat":41.256341,"lon":-72.99019},{"id":24,"name":"Building 750","description":"","lat":41.256796,"lon":-72.988118},{"id":25,"name":"Building 800","description":"","lat":41.257929,"lon":-72.989021},{"id":26,"name":"Building 900","description":"","lat":41.260015,"lon":-72.986981},{"id":27,"name":"Canal / Munson","description":"","lat":41.321292,"lon":-72.929656},{"id":28,"name":"Canner / Livingston","description":"","lat":41.323937,"lon":-72.913996},{"id":29,"name":"Canner / Whitney (N)","description":"","lat":41.32462,"lon":-72.916166},{"id":30,"name":"Chapel / Church","description":"","lat":41.305963,"lon":-72.925461},{"id":31,"name":"Chapel / College","description":"","lat":41.30724,"lon":-72.928355},{"id":32,"name":"Chapel / State Elm City Market","description":"","lat":41.304876,"lon":-72.923138},{"id":33,"name":"Chapel / York","description":"","lat":41.30842,"lon":-72.931341},{"id":34,"name":"Chemistry / 225 Prospect","description":"","lat":41.318492,"lon":-72.923687},{"id":35,"name":"Church / George","description":"","lat":41.30342,"lon":-72.92784},{"id":36,"name":"Church / Grove","description":"","lat":41.310242,"lon":-72.922595},{"id":37,"name":"Church Wall","description":"","lat":41.309632,"lon":-72.923067},{"id":38,"name":"College / Crown","description":"","lat":41.306177,"lon":-72.929592},{"id":39,"name":"College / George","description":"","lat":41.305293,"lon":-72.930255},{"id":40,"name":"College / Grove (N)","description":"","lat":41.311805,"lon":-72.925425},{"id":41,"name":"College / Wall (N)","description":"","lat":41.310583,"lon":-72.926188},{"id":42,"name":"College / Wall (S)","description":"","lat":41.310836,"lon":-72.926148},{"id":43,"name":"Congress / Cedar","description":"","lat":41.30199,"lon":-72.933299},{"id":44,"name":"Congress / Howard","description":"","lat":41.301495,"lon":-72.935029},{"id":45,"name":"Court / Olive","description":"","lat":41.305197,"lon":-72.920117},{"id":46,"name":"Davenport / Howard","description":"","lat":41.303096,"lon":-72.936589},{"id":47,"name":"Divinity / 409 Prospect","description":"","lat":41.323538,"lon":-72.923285},{"id":48,"name":"Division / Prospect","description":"","lat":41.324769,"lon":-72.923522},{"id":49,"name":"Division / Sheffield","description":"","lat":41.325324,"lon":-72.926572},{"id":50,"name":"Edwards / Orange","description":"","lat":41.318086,"lon":-72.914395},{"id":51,"name":"Elm / High","description":"","lat":41.310194,"lon":-72.928972},{"id":52,"name":"Elm / York","description":"","lat":41.31049,"lon":-72.930123},{"id":53,"name":"Elm / York (TYCO)","description":"","lat":41.311005,"lon":-72.930893},{"id":54,"name":"Elm College","description":"","lat":41.309579,"lon":-72.927476},{"id":55,"name":"Foster / Avon","description":"","lat":41.319738,"lon":-72.910418},{"id":56,"name":"Foster / Cottage","description":"","lat":41.318998,"lon":-72.910985},{"id":57,"name":"Foster / Edwards","description":"","lat":41.317265,"lon":-72.912309},{"id":58,"name":"Foster / Lawrence","description":"","lat":41.318154,"lon":-72.911633},{"id":59,"name":"Front / Rt 1 (N)","description":"","lat":41.296105,"lon":-72.955812},{"id":60,"name":"Front / Rt 1 (S)","description":"","lat":41.296071,"lon":-72.955927},{"id":61,"name":"Front Street South","description":"","lat":41.296158,"lon":-72.956009},{"id":62,"name":"Grove / College","description":"","lat":41.311698,"lon":-72.925318},{"id":63,"name":"Grove / Temple","description":"","lat":41.311134,"lon":-72.924022},{"id":64,"name":"Grove / Whitney","description":"","lat":41.310421,"lon":-72.922331},{"id":65,"name":"Grove and Hillhouse","description":"","lat":41.311218,"lon":-72.924262},{"id":66,"name":"Helen Hadley Hall","description":"","lat":41.312459,"lon":-72.922824},{"id":67,"name":"Wall / York","description":"","lat":41.312102,"lon":-72.928948},{"id":68,"name":"Humphrey / Orange","description":"","lat":41.316218,"lon":-72.916544},{"id":69,"name":"Huntington / Edgehill (E)","description":"","lat":41.329965,"lon":-72.917606},{"id":70,"name":"Huntington / Edgehill (W)","description":"","lat":41.330004,"lon":-72.917272},{"id":71,"name":"Lanman","description":"","lat":41.314967,"lon":-72.929998},{"id":72,"name":"LEPH / 60 College","description":"","lat":41.303422,"lon":-72.931698},{"id":73,"name":"Library Walk","description":"","lat":41.309479,"lon":-72.930752},{"id":74,"name":"Lot 22 - Whitney / Humphrey","description":"","lat":41.317907,"lon":-72.920416},{"id":75,"name":"Olive / Chapel","description":"","lat":41.304125,"lon":-72.920274},{"id":76,"name":"Orange / Audobon","description":"","lat":41.310923,"lon":-72.920191},{"id":77,"name":"Orange / Avon","description":"","lat":41.320428,"lon":-72.912916},{"id":78,"name":"Orange / Bishop (N)","description":"","lat":41.316978,"lon":-72.915521},{"id":79,"name":"Orange / Bishop (S)","description":"","lat":41.317355,"lon":-72.915423},{"id":80,"name":"Orange / Bradley (N)","description":"","lat":41.31298,"lon":-72.918548},{"id":81,"name":"Orange / Bradley (S)","description":"","lat":41.313013,"lon":-72.918668},{"id":82,"name":"Orange / Canner","description":"","lat":41.323303,"lon":-72.910898},{"id":83,"name":"Orange / Cottage","description":"","lat":41.320062,"lon":-72.913347},{"id":84,"name":"Orange / Edwards (N)","description":"","lat":41.317957,"lon":-72.914793},{"id":85,"name":"Orange / Edwards (S)","description":"","lat":41.318385,"lon":-72.914615},{"id":86,"name":"Orange / Grove","description":"","lat":41.310003,"lon":-72.920939},{"id":87,"name":"Orange / Humphrey (N)","description":"","lat":41.315906,"lon":-72.916327},{"id":88,"name":"Orange / Humphrey (S)","description":"","lat":41.316338,"lon":-72.916176},{"id":89,"name":"Orange / Lawrence (N)","description":"","lat":41.31883,"lon":-72.91412},{"id":90,"name":"Orange / Lawrence (S)","description":"","lat":41.319282,"lon":-72.913902},{"id":91,"name":"Orange / Pearl (N)","description":"","lat":41.314515,"lon":-72.917391},{"id":92,"name":"Orange / Pearl (S)","description":"","lat":41.314826,"lon":-72.917319},{"id":93,"name":"Orange / Trumbull","description":"","lat":41.312323,"lon":-72.919177},{"id":94,"name":"Orange / Willow (N)","description":"","lat":41.321894,"lon":-72.911811},{"id":95,"name":"Orange / Willow (S)","description":"","lat":41.32224,"lon":-72.911692},{"id":96,"name":"Payne Whitney Gym","description":"","lat":41.313444,"lon":-72.930694},{"id":97,"name":"Peabody Museum / Whitney / Sachem","description":"","lat":41.315675,"lon":-72.920859},{"id":98,"name":"Phelps Gate","description":"","lat":41.308356,"lon":-72.927978},{"id":99,"name":"Pierson Sage Garage","description":"","lat":41.319394,"lon":-72.919091},{"id":100,"name":"Prospect / Canner","description":"","lat":41.325351,"lon":-72.922891},{"id":101,"name":"Prospect / Edwards","description":"","lat":41.320038,"lon":-72.923299},{"id":102,"name":"Prospect / Highland (N)","description":"","lat":41.328261,"lon":-72.921951},{"id":103,"name":"Prospect / Highland (S)","description":"","lat":41.328269,"lon":-72.92208},{"id":104,"name":"Prospect / Hillside","description":"","lat":41.321076,"lon":-72.923422},{"id":105,"name":"Prospect / Huntington","description":"","lat":41.330366,"lon":-72.921482},{"id":106,"name":"Prospect / Sachem (N)","description":"","lat":41.315749,"lon":-72.924393},{"id":107,"name":"Prospect / Sachem (S)","description":"","lat":41.315795,"lon":-72.924541},{"id":108,"name":"Prospect / Trumbull","description":"","lat":41.314021,"lon":-72.924894},{"id":109,"name":"Quigley Stadium Inbound","description":"","lat":41.293932,"lon":-72.955238},{"id":110,"name":"Quigley Stadium Outbound","description":"","lat":41.293438,"lon":-72.954994},{"id":111,"name":"Sachem / Whitney","description":"","lat":41.315444,"lon":-72.921229},{"id":112,"name":"Sachem / Winchester","description":"","lat":41.316387,"lon":-72.92649},{"id":113,"name":"SCL","description":"","lat":41.318189,"lon":-72.923912},{"id":114,"name":"SOM","description":"","lat":41.315224,"lon":-72.920887},{"id":115,"name":"State St Station","description":"","lat":41.30519,"lon":-72.922331},{"id":116,"name":"Stop & Shop","description":"","lat":41.315041,"lon":-72.938202},{"id":117,"name":"Gilbert / Cedar","description":"","lat":41.301613,"lon":-72.933459},{"id":118,"name":"Temple / Grove","description":"","lat":41.311184,"lon":-72.923753},{"id":119,"name":"Trader Joe's","description":"","lat":41.251375,"lon":-73.018082},{"id":120,"name":"Trumbull / Hillhouse","description":"","lat":41.3135,"lon":-72.923286},{"id":121,"name":"Union Station (N)","description":"","lat":41.297857,"lon":-72.926763},{"id":122,"name":"Union Station (S)","description":"","lat":41.297929,"lon":-72.926912},{"id":123,"name":"VA Entrance Inbound","description":"","lat":41.284966,"lon":-72.958144},{"id":124,"name":"VA Entrance Outbound","description":"","lat":41.284845,"lon":-72.957845},{"id":125,"name":"VA Hospital","description":"","lat":41.281934,"lon":-72.96194},{"id":126,"name":"Wall / Church","description":"","lat":41.309552,"lon":-72.923477},{"id":127,"name":"West Haven Train Station","description":"","lat":41.271172,"lon":-72.963517},{"id":128,"name":"Whitney / Audubon","description":"","lat":41.311557,"lon":-72.922197},{"id":129,"name":"Whitney / Canner","description":"","lat":41.324692,"lon":-72.916817},{"id":130,"name":"Whitney / Cold Spring (S)","description":"","lat":41.326294,"lon":-72.915854},{"id":131,"name":"Whitney / Cold Springs (N)","description":"","lat":41.325918,"lon":-72.915845},{"id":132,"name":"Whitney / Cottage (N)","description":"","lat":41.321181,"lon":-72.918038},{"id":133,"name":"Whitney / Cottage (S)","description":"","lat":41.321281,"lon":-72.918194},{"id":134,"name":"Whitney / Edwards (N)","description":"","lat":41.31976,"lon":-72.918722},{"id":135,"name":"Whitney / Edwards (S)","description":"","lat":41.319793,"lon":-72.918903},{"id":136,"name":"Whitney / Highland","description":"","lat":41.328281,"lon":-72.914879},{"id":137,"name":"Whitney / Humphrey (N)","description":"","lat":41.317542,"lon":-72.919779},{"id":138,"name":"Whitney / Humphrey (S)","description":"","lat":41.318019,"lon":-72.919756},{"id":139,"name":"Whitney / Huntington","description":"","lat":41.329605,"lon":-72.914275},{"id":140,"name":"Whitney / Linden","description":"","lat":41.323205,"lon":-72.917287},{"id":141,"name":"Whitney / Trumbull","description":"","lat":41.313259,"lon":-72.92169},{"id":142,"name":"Willow / Foster","description":"","lat":41.321121,"lon":-72.909616},{"id":143,"name":"Willow / Livingston","description":"","lat":41.32299,"lon":-72.913872},{"id":144,"name":"Willow / Whitney","description":"","lat":41.323779,"lon":-72.916575},{"id":145,"name":"Science Park Garage","description":"","lat":41.323328,"lon":-72.928795},{"id":146,"name":"Winchester / Division","description":"","lat":41.325379,"lon":-72.92774},{"id":147,"name":"Winchester / Sachem","description":"","lat":41.316131,"lon":-72.926506},{"id":148,"name":"Winchester / Woodland","description":"","lat":41.319277,"lon":-72.927761},{"id":149,"name":"York / Cedar","description":"","lat":41.303784,"lon":-72.935017},{"id":150,"name":"York / Chapel","description":"","lat":41.308511,"lon":-72.931505},{"id":151,"name":"York / Crown","description":"","lat":41.30702,"lon":-72.93265}]

# def find_progress(lat, lon, points=points, distances=distances, total_distance=total_distances):

route_stops_dict = {}

for stop in route_stops:
    for stop_info in stops_info:
        if stop_info['id'] == stop:
            progress_of_stop, _ = find_progress(stop_info['lat'], stop_info['lon'])
            route_stops_dict[stop] = progress_of_stop
            break
    
print(route_stops_dict)

{10: 0.7574379285320636, 2: 0.8100693903319043, 5: 0.8440054213568458, 52: 0.8716423053862119, 41: 0.5939719172388357, 20: 0.9561084949286741, 108: 0.9760129679929708, 106: 1.0, 34: 0.038237900811054396, 101: 0.05962329244285599, 47: 0.10745151555996796, 100: 0.1328101369610443, 102: 0.1737593212303774, 105: 0.2030971029708586, 69: 0.24730806822917184, 139: 0.2835216542743249, 136: 0.30136998545272076, 130: 0.33057193150829534, 129: 0.35416926058718823, 140: 0.3765943938182113, 133: 0.4042609751799648, 135: 0.42607808057732677, 138: 0.4518605417805998, 97: 0.4856001998816069, 66: 0.5344474411127405, 63: 0.558828997582244, 42: 0.5908789960154363, 98: 0.6298985138882538, 38: 0.663908526212489, 39: 0.677592282664344, 72: 0.7073775345450428, 43: 0.7346760880904396}
